# SocietyOfMindAgent

This notebook demonstrates the SocietyOfMindAgent, which runs a group chat as an internal monologue, but appears to the external world as a single agent. This confers three distinct advantages:

1. It provides a clean way of producing a hierarchy of agents, hiding complexity as inner monologues.
2. It provides a consistent way of extracting an answer from a lengthy group chat (normally, it is not clear which message is the final response, and the response itself may not always be formatted in a way that makes sense when extracted as a standalone message).
3. It provides a way of recovering when agents exceed their context window constraints (the inner monologue is protected by try-catch blocks)

````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

In [1]:
%cd ..

/workspaces/cv-optimizer


/usr/local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [15]:
from utils.filestore import get_completed_cv_data,get_cv_blueprint,get_position_data,set_position_cv_offers
import json
position_data = get_position_data()
cv_data = get_completed_cv_data()
cv_blueprint = get_cv_blueprint()

In [24]:
from typing_extensions import Annotated
import os

from dotenv import load_dotenv

load_dotenv()

import autogen

llm_config = {"model": "gpt-3.5-turbo", "api_key":os.environ['OPENAI_API_KEY'] , "cache_seed": 42}

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

### Example Group Chat with Two Agents

In this example, we will use an AssistantAgent and a UserProxy agent (configured for code execution) to work together to solve a problem. Executing code requires *at least* two conversation turns (one to write the code, and one to execute the code). If the code fails, or needs further refinement, then additional turns may also be needed. When will then wrap these agents in a SocietyOfMindAgent, hiding the internal discussion from other agents (though will still appear in the console), and ensuring that the response is suitable as a standalone message.

#### Construct the Inner-Monologue Agents
We begin by constructing the inner-monologue agents. These are the agents that do that real work.

In [25]:
writer = autogen.AssistantAgent(
    llm_config= config_list,
    name="External_recruiter",
    description="external HR recruit that will help me get the position,",
    system_message=f"""
    You are an independent HR recruiter, committed to referring the perfect candidate for the job. 
    You help candidates to optimize the CV for the position, optimize the CV and output it in the following format:
    ```json
    {json.dumps(cv_blueprint,indent=4)}
    ```
    You must not fabricate information! 
    If you find some requirements in the position that you assume will prevent the user CV's to be accepted for an interview, you can ask the user about their experience only, you can use ask_human_expert function.
    """,
)
critic = autogen.AssistantAgent(
    name= "Hiring_technical_recruiter",
    description="technical recruiter in the hiring company",
    system_message=f"""
        You are an experienced technical recruiter tasked with filling an open position in a leading tech company. 
        Your goal is to find the best candidate who not only possesses the necessary technical skills but 
        also fits well with the company culture.  Your Job is to give critical feedback on the CV you receive to the position.
        Be concise, professional, and engaging in your communication.

        The position you are hiring is:
        {json.dumps(position_data,indent=4)}
        """,
    llm_config= config_list
    
)

groupchat = autogen.GroupChat(
    agents=[writer, critic],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
)

#### Construct and Run the SocietyOfMind Agent
We now wrap the inner group-chat with the SocietyOfMind Agent, and create a UserProxy to talk to it.

In [26]:
from autogen.agentchat.contrib.society_of_mind_agent import SocietyOfMindAgent  # noqa: E402

society_of_mind_agent = SocietyOfMindAgent(
    "society_of_mind",
    chat_manager=manager,
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
    description="The User that would like to submit its CV",
    system_message=f"""You are looking to get an interview, your CV is:
        {json.dumps(cv_data,indent=4)}
        """
)

user_proxy.initiate_chat(society_of_mind_agent, message=f"""
Hey there,
I've come across this amazing job opportunity that I'm excited about, and I want to make sure my CV is perfectly tailored to it. I've attached the job description below so you can get a sense of what they're looking for.

Could you please review my CV and make any necessary adjustments to better align it with the job description? I want to make sure I highlight the relevant skills and experiences without making it obvious that I've optimized it. 
Also, please make sure not to add any information that isn't already in my CV.

Thanks so much for your help, I really appreciate it!
My CV:
{json.dumps(cv_data,indent=4)}
                        
Position Description:
{json.dumps(position_data,indent=4)}
""")

user_proxy (to society_of_mind):


Hey there,
I've come across this amazing job opportunity that I'm excited about, and I want to make sure my CV is perfectly tailored to it. I've attached the job description below so you can get a sense of what they're looking for.

Could you please review my CV and make any necessary adjustments to better align it with the job description? I want to make sure I highlight the relevant skills and experiences without making it obvious that I've optimized it. 
Also, please make sure not to add any information that isn't already in my CV.

Thanks so much for your help, I really appreciate it!
My CV:
{
    "personal_info": {
        "firstname": "Sefi",
        "lastname": "Erlich",
        "email": "Berlichsefi@gmail.com",
        "phone": "+972 524 307 093",
        "address": "TLV, Israel - willing to relocate",
        "linkedin": {
            "link": "https://www.linkedin.com/in/erlichsefi/",
            "username": []
        },
        "github": {


Traceback (most recent call last):
  File "/root/.local/lib/python3.11/site-packages/autogen/agentchat/contrib/society_of_mind_agent.py", line 192, in generate_inner_monologue_reply
    self.initiate_chat(self.chat_manager, message=messages[-1], clear_history=False)
  File "/root/.local/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py", line 991, in initiate_chat
    self.send(msg2send, recipient, silent=silent)
  File "/root/.local/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py", line 632, in send
    recipient.receive(message, self, request_reply, silent)
  File "/root/.local/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py", line 792, in receive
    reply = self.generate_reply(messages=self.chat_messages[sender], sender=sender)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/.local/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py", line 1921, in generate_re

ChatResult(chat_id=None, chat_history=[{'content': '\nHey there,\nI\'ve come across this amazing job opportunity that I\'m excited about, and I want to make sure my CV is perfectly tailored to it. I\'ve attached the job description below so you can get a sense of what they\'re looking for.\n\nCould you please review my CV and make any necessary adjustments to better align it with the job description? I want to make sure I highlight the relevant skills and experiences without making it obvious that I\'ve optimized it. \nAlso, please make sure not to add any information that isn\'t already in my CV.\n\nThanks so much for your help, I really appreciate it!\nMy CV:\n{\n    "personal_info": {\n        "firstname": "Sefi",\n        "lastname": "Erlich",\n        "email": "Berlichsefi@gmail.com",\n        "phone": "+972 524 307 093",\n        "address": "TLV, Israel - willing to relocate",\n        "linkedin": {\n            "link": "https://www.linkedin.com/in/erlichsefi/",\n            "use

#### Remarks

There are a few things to notice about this output:
- First, the user_proxy sent only one message to the society_of_mind agent, and received only one message in response. As far as it is concerned, the society_of_mind agent is the only agent in the chat.
- Second, the final response is formatted in a way that is standalone. Unlike the prior response, it makes no reference of a previous script or execution, and it lacks the TERMINATE keyword that ended the inner monologue.